In [9]:
from shapely import from_wkt
import polars as pl
from polars import col as c
from shapely.geometry import Point


In [13]:
points

geo
list[str]
"[""POINT (0 0)"", ""POINT (1 0)"", ""POINT (2 0)""]"
"[""POINT (0 1)"", ""POINT (1 1)"", ""POINT (2 1)""]"
"[""POINT (0 2)"", ""POINT (1 2)"", ""POINT (2 2)""]"


In [20]:

# Generate a list of lists of points
points = pl.DataFrame({"geo": [[(Point(x, y)).wkt for x in range(3)] for y in range(3)]})

points.select(
    c("geo").list.eval(pl.element().map_elements(from_wkt, return_dtype=pl.Object))
)

thread '<unnamed>' panicked at crates/polars-core/src/series/iterator.rs:84:9:
object dtype not supported in Series.iter


PanicException: object dtype not supported in Series.iter